In [1]:
from scipy import stats

#------- FUNCTIONS ----------

# Power function
def get_power(n, p1, p2, cl):
    alpha = 1 - cl
    qu = stats.norm.ppf(1 - alpha/2)
    diff = abs(p2-p1)
    bp = (p1+p2) / 2

    v1 = p1 * (1-p1)
    v2 = p2 * (1-p2)
    bv = bp * (1-bp)

    power_part_one = stats.norm.cdf((n**0.5 * diff - qu * (2 * bv)**0.5) / (v1+v2) ** 0.5)
    power_part_two = 1 - stats.norm.cdf((n**0.5 * diff + qu * (2 * bv)**0.5) / (v1+v2) ** 0.5)

    power = power_part_one + power_part_two

    return (power)

# Get Sample Size function
def get_sample_size(power, p1, p2, cl, max_n=1000000):
    n = 1
    while n <= max_n:
        tmp_power = get_power(n, p1, p2, cl)

        if tmp_power >= power:
            return n
        else:
            n = n + 100

    return "Increase Max N Value"

# P-value function
def get_pvalue(con_conv, test_conv, con_size, test_size):
    lift = -abs(test_conv - con_conv)

    scale_one = con_conv * (1-con_conv) * (1/ con_size)
    scale_two = test_conv * (1-test_conv) * (1/ test_size)
    scale_val = (scale_one + scale_two) ** 0.5

    p_value = 2 * stats.norm.cdf(lift, loc=0, scale = scale_val)
    return p_value

# Confidence Interval function
def get_ci(lift, alpha, sd):
    val = abs(stats.norm.ppf((1-alpha)/2))

    lwr_bnd = lift - val * sd
    upr_bnd = lift + val * sd

    return (lwr_bnd, upr_bnd)


In [6]:
#------ EXAMPLE ------#

#----- INPUT USER -------
testing_metric = "% clientes con ordenes"
baseline = 0.025
lift = 0.20

#----- SAMPLE SIZE CALCULATION -----
p2 = baseline * (1+lift)
power = 0.8
CL = 0.9

test_sample = get_sample_size(power, baseline, p2, CL)

print(baseline)
print(p2)
print(test_sample)

0.025
0.03
13301


In [8]:
#------- TEST CONVERTION RATE -------#

#----- INPUT USER ------

# Treatment

sample_size_treatment = 15000
metric_baseline_treatment = 0.0246
metric_post_treatment = 0.0301

# Control

sample_size_control = 15000
metric_baseline_control = 0.0243
metric_post_control = 0.026

#---- P-VALUE, TEST POWER & CL CALCULATIONS -----

con_conv =  (metric_post_control / metric_baseline_control) - 1
test_conv = (metric_post_treatment / metric_baseline_treatment) - 1
con_size = sample_size_control
test_size = sample_size_treatment

lift_mean = test_conv - con_conv
lift_variance = (1 - test_conv) * test_conv /test_size + (1 - con_conv) * con_conv / con_size
lift_sd = lift_variance**0.5

p_value = get_pvalue(con_conv, test_conv, con_size, test_size)
CL_test = get_ci(lift_mean, 0.95, lift_sd)

test_power = get_power(sample_size_treatment, metric_baseline_treatment, metric_post_treatment, 0.9)

print(test_power)
print(p_value)
print(CL_test)

0.8989694837281451
0.0
(0.1458005298635427, 0.16143624620792135)
